In [42]:
from PIL import Image
from PIL import ImageDraw
import timeit
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from skimage.feature import greycomatrix, greycoprops
import matplotlib.image as mpimg 
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import random
from PIL import ImageEnhance
import csv

In [43]:
map_8bit_to_3bit = [i // 32 for i in range(256)]
predicts=[]
imgs = {}
imgs["cap"] = []
imgs["tulis"] = []
val_imgs = {}
val_imgs["cap"] = []
val_imgs["tulis"] = []
model = {}
model_test = {}
neighbor = 3
time_modelling_start = timeit.default_timer()
model["cap"] = []
model["tulis"] = []
model_test["cap"] = []
model_test["tulis"] = []
mean = {}
variance = {}
mean["tulis"] = []
variance["tulis"] = []
mean["cap"] = []
variance["cap"] = []
a1 = 0
b1 = 0

In [44]:
def reset():
    global map_8bit_to_3bit
    global model
    global model_test
    global time_modelling_start
    global mean
    global variance
    global a1
    global b1
    map_8bit_to_3bit = [i // 32 for i in range(256)]
    model = {}
    model_test = {}
    neighbor = 3
    time_modelling_start = timeit.default_timer()
    model["cap"] = []
    model["tulis"] = []
    model_test["cap"] = []
    model_test["tulis"] = []
    mean = {}
    variance = {}
    mean["tulis"] = []
    variance["tulis"] = []
    mean["cap"] = []
    variance["cap"] = []
    a1 =0
    b1=0
    predicts=[]

In [45]:
#Pre Processing
def load_img(img_path):
    return Image.open(img_path).convert('L')

def print_img(img):
    plt.imshow(img)
    
def get_img_colors(img):
    return list(img.getdata())


def get_3bit_colors(img):
    img_colors = get_img_colors(img)
    loop_count = 0
    
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = [] 
    loop_count = 0
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

In [46]:
#Features Extraction
def feature_extract(img, sampling_count, class_name):
    glcm_component=[]
    img_3bit_colors = get_3bit_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    glcm_matrix=greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    component = ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']
    for x in component:
        glcm_component.append(greycoprops(glcm_matrix, x)[0][0])
    glcm_component.append(class_name)
    return(glcm_component)

In [47]:
#Modelling
def get_class_names(training_folder_path):
    return os.listdir(training_folder_path)

def make_a_model(class_name, features, learning_rate):
    random_splitter = random.uniform(0, 1)
    if(random_splitter <= learning_rate):
        model[class_name].append(features)
    else:
        model_test[class_name].append(features)
        
def to_dataframe(model):
    models = pd.DataFrame(model['cap'] + model['tulis'], columns=['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity', 'class'])
    return models

In [48]:
#Classification
def naive_bayes(image, cap_probability, tulis_probability):
    prob = 1
    probability = 0.00
    for class_name in class_names:
        if(class_name=="tulis"):
            prob=tulis_probability
        elif(class_name=="cap"):
            prob=cap_probability
        for i in range(6):
            a = (image[i]-mean[class_name][i])**2/(2*variance[class_name][i])
            b = 1/(np.sqrt(2*3.14*variance[class_name][i]))
            prob=prob * (b* np.exp(0-a))
       
        if(prob>probability):
            probability = prob
            probability_class = class_name
        prob=1
    return probability_class

In [49]:
#Testing

def testing(cap_probability, tulis_probability):
    right = 0
    total = 0
    for class_name in class_names:
        for image in model_test[class_name]:
            predict = naive_bayes(image, cap_probability, tulis_probability)
            class_image = class_name
            if(predict == class_name):
                   right+=1
            total +=1
    accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) +"%\n")
    
    
def testing_out(sampling_count):
    testing_img_paths = glob.glob(test_folder_path + '/' + img_type)
    for testing_img_path in testing_img_paths:
        imgs = load_img(testing_img_path)
        predict_class = knn(imgs, sampling_count, neighbor)
        print("Categorized as " + predict_class)
        plt.imshow(Image.open(testing_img_path))    

In [50]:
def testings():
    global a1, b1
    total={}
    total["tulis"] = models.loc[models['class'] == 'tulis']
    total["cap"] = models.loc[models['class'] == 'cap']
    mean["tulis"] = []
    variance["tulis"] = []
    mean["cap"] = []
    variance["cap"] = []
    tulis_probability = (total["tulis"].count()/models.count())[0]
    cap_probability = (total["cap"].count()/models.count())[0]
    a1=tulis_probability
    b1=cap_probability
    for a in ["tulis", "cap"]:
        for b in ['contrast', 'homogeneity', 'energy', 'correlation', 'ASM', 'dissimilarity']:
            mean[a].append(total[a][b].mean())
            variance[a].append(total[a][b].var())
    
    testing(cap_probability, tulis_probability)

In [51]:
#Validation
def validate(sampling_count, a, b, val_image_features, iterate, epoch):
    
    fp_tulis, fn_tulis, tp_tulis, tn_tulis = 0,0,0,0
    fp_cap, fn_cap, tp_cap, tn_cap = 0,0,0,0
    right = 0
    total = 0
    for class_name in class_names:
        for image in val_image_features[class_name]:
            predict = naive_bayes(image,a,b)
            predicts.append(predict)
            class_image = class_name
            if(predict == class_name):
                right+=1

            total +=1
        
        accuracy = (right/total) * 100
    print("Accuracy: " + str(accuracy) + " "+ str(right) + "%\n")
    peformance(iterate, epoch)

def peformance(iterate, epoch):
    
    fp_tulis, fn_tulis, tp_tulis, tn_tulis = 0,0,0,0
    fp_cap, fn_cap, tp_cap, tn_cap = 0,0,0,0

    for class_name in class_names:
        for i in range(118): #Class = Cap
            if(predicts[i] == "cap"):
                if(class_name == "cap"):
                    tp_cap+=1
                else:
                    tn_tulis+=1
            else:
                if(class_name == "cap"):
                    fp_cap+=1
                else:
                    fn_tulis+=1

        for j in range(i+1, 256): #Class = Tulis
            if(predicts[j] == "tulis"):
                if(class_name == "cap"):
                    tn_cap+=1
                else:
                    tp_tulis+=1
            else:
                if(class_name == "cap"):
                    fn_cap+=1
                else:
                    fp_tulis+=1

    recall_tulis = round(((tp_tulis / (tp_tulis + fn_tulis))*100),4)
    precission_tulis = round(((tp_tulis / (tp_tulis + fp_tulis))*100),4)
    f1_score_tulis =  2 * (recall_tulis*precission_tulis) / (recall_tulis + precission_tulis)

    recall_cap = round(((tp_cap / (tp_cap + fn_cap))*100),4)
    precission_cap = round(((tp_cap / (tp_cap + fp_cap))*100),4)
    f1_score_cap=  2 * (recall_cap*precission_cap) / (recall_cap + precission_cap)
    
    recall = (recall_cap+recall_tulis)/2
    precission = (precission_cap + precission_tulis)/2
    f1_score = (f1_score_cap + f1_score_tulis)/2
    
 
    csvRow = ["class", "Recall", "Precission", "F1-Score"]
    csvfile = "Peformance/"+str(epoch+1)+"_"+str(iterate+1)+".csv"
    with open(csvfile, "w") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(csvRow)
        wr.writerow(['cap', recall_cap, precission_cap, f1_score_cap])
        wr.writerow(['tulis', recall_tulis, precission_tulis, f1_score_tulis])
        wr.writerow(['avg/total', recall, precission, f1_score])

In [55]:
learning_rate = 0.8
sampling_count = 128
img_type = '*.jpg'
root_path = './'
training_folder_path = root_path + 'glcm/training/'
validation_folder_path = root_path + 'glcm/validation/'
test_folder_path = root_path + 'glcm/test/'

class_names = get_class_names(training_folder_path)

for class_name in class_names:
    training_img_paths = glob.glob(training_folder_path + class_name + '/' + img_type)
    validate_img_paths = glob.glob(validation_folder_path + class_name + '/' +img_type)
    
    for training_img_path in training_img_paths:
        training_img = load_img(training_img_path)
        enhancer = ImageEnhance.Sharpness(training_img)
        training_img = enhancer.enhance(1.5)
        training_img_class_name = class_name
        feature = feature_extract(training_img, sampling_count, training_img_class_name)
        imgs[class_name].append(feature)
        
    for validate_img_path in validate_img_paths:
            img = load_img(validate_img_path)
            enhancer = ImageEnhance.Sharpness(img)
            img = enhancer.enhance(1.5)
            validate_img_class_name = class_name
            val_image_feature = feature_extract(img, sampling_count, validate_img_class_name)
            val_imgs[class_name].append(val_image_feature)
    


for j in range(5):
    for i in range(50):
        for class_name in class_names:
            for feature in imgs[class_name]:
                make_a_model(class_name, feature, learning_rate)

        models = to_dataframe(model)
        models_test = to_dataframe(model_test)
        print("Epoch ke-"+ str(i+1) + " ")
        testings()
        
    print("Iterasi ke-"+ str(j+1) + " ")
    validate(128,a1,b1, val_imgs, j, i)
    time_modelling_stop = timeit.default_timer()
    print('Time elapsed to make this model is ' + str(time_modelling_stop - time_modelling_start) + ' seconds.')
    reset()
imgs["cap"] = []
imgs["tulis"] = []
val_imgs["cap"] = []
val_imgs["tulis"] = []

Epoch ke-1 
Accuracy: 83.33333333333334%

Epoch ke-2 
Accuracy: 78.0%

Epoch ke-3 
Accuracy: 78.18791946308725%

Epoch ke-4 
Accuracy: 78.43137254901961%

Epoch ke-5 
Accuracy: 79.80392156862746%

Epoch ke-6 
Accuracy: 78.64077669902912%

Epoch ke-7 
Accuracy: 77.48976807639836%

Epoch ke-8 
Accuracy: 77.38095238095238%

Epoch ke-9 
Accuracy: 77.39221871713985%

Epoch ke-10 
Accuracy: 77.56833176248821%

Epoch ke-11 
Accuracy: 77.59656652360515%

Epoch ke-12 
Accuracy: 76.55880031570639%

Epoch ke-13 
Accuracy: 76.48335745296671%

Epoch ke-14 
Accuracy: 76.94915254237287%

Epoch ke-15 
Accuracy: 76.83080808080808%

Epoch ke-16 
Accuracy: 76.61721068249258%

Epoch ke-17 
Accuracy: 76.53631284916202%

Epoch ke-18 
Accuracy: 77.12933753943217%

Epoch ke-19 
Accuracy: 77.18859429714857%

Epoch ke-20 
Accuracy: 77.0764119601329%

Epoch ke-21 
Accuracy: 76.8018018018018%

Epoch ke-22 
Accuracy: 77.02644126571305%

Epoch ke-23 
Accuracy: 77.00247729149463%

Epoch ke-24 
Accuracy: 76.984126984

Accuracy: 76.54473554127533%

Epoch ke-38 
Accuracy: 76.92492781520693%

Epoch ke-39 
Accuracy: 76.93935119887165%

Epoch ke-40 
Accuracy: 76.65137614678899%

Epoch ke-41 
Accuracy: 76.64118705035972%

Epoch ke-42 
Accuracy: 76.69732807709154%

Epoch ke-43 
Accuracy: 76.74716819833299%

Epoch ke-44 
Accuracy: 76.71261487050961%

Epoch ke-45 
Accuracy: 76.73753066230582%

Epoch ke-46 
Accuracy: 76.96921231507396%

Epoch ke-47 
Accuracy: 77.0443925233645%

Epoch ke-48 
Accuracy: 77.21712538226299%

Epoch ke-49 
Accuracy: 77.20119895091796%

Epoch ke-50 
Accuracy: 77.22609014290948%

Iterasi ke-4 
Accuracy: 75.0 192%

Time elapsed to make this model is 10.003173699999934 seconds.
Epoch ke-1 
Accuracy: 73.04347826086956%

Epoch ke-2 
Accuracy: 72.14611872146118%

Epoch ke-3 
Accuracy: 74.375%

Epoch ke-4 
Accuracy: 76.55172413793103%

Epoch ke-5 
Accuracy: 76.13019891500905%

Epoch ke-6 
Accuracy: 76.03053435114504%

Epoch ke-7 
Accuracy: 75.26315789473685%

Epoch ke-8 
Accuracy: 75.830469